In [1]:
def fft_gauss(sigma):
    length=wavelen
    w = empty(length)
    out = empty(length)
    input_ = zeros(length)
    input_[int(length/2)]=1.
    if sigma>=2.5:
        q = 0.98711*sigma-0.96330
    else:
        if sigma>=0.5:
            q = 3.97156-4.14554*sqrt(1-0.26891*sigma)
        else:
            q = 3.97156 - 4.14554*sqrt(1-0.26891*sigma);
    b0 = 1.57825+2.44413*q+1.4281*q*q+0.422205*q*q*q;
    b1 = 2.44413*q+2.85619*q*q+1.26661*q*q*q;
    b2 = -1.*(1.4281*q*q+1.26661*q*q*q);
    b3 = 0.422205*q*q*q;
    B = 1.-(b1+b2+b3)/b0;
    for i in range(length):
        w[i] = B*input_[i]
        if i>=3:
            w[i] += b3*w[i-3]/b0
        if i>=2:
            w[i] += b2*w[i-2]/b0
        if i>=1:
            w[i] += b1*w[i-1]/b0
    for i in range(length-1,-1,-1):
        out[i] = B*w[i]
        if i<=length-2:
            out[i] += b1*out[i+1]/b0
        if i<=length-3:
            out[i] += b2*out[i+2]/b0
        if i<=length-4:
            out[i] += b3*out[i+3]/b0
    for i in range(int(length/2)):
        out[i] = out[i+int(length/2)]
    for i in range(int(length/2)):
        out[length-1-i] = out[i]
    res = fft.rfft(out)
    return res;

In [2]:
def fft_cusp(shape_time, tau):
    length=wavelen
    k=2*shape_time+1;
    length = length+k;  
    m2=1.; 
    m1=m2/(exp(1./tau)-1.);
    p = empty(length)
    q = empty(length)
    s = empty(length)
    input_ = zeros(length)
    p[0]=q[0]=s[0]=input_[0]=0.
    input_[1]=1.
    dk=0.
    dl=0.
    for i in range(1,length):
        dk=0.
        dl=0.
        dk = input_[i]
        if i>=k:
            dk -= input_[i-k]

        if i>=shape_time:
            dl += input_[i-shape_time]

        if i>=shape_time+1:
            dl -= input_[i-shape_time-1]

        
        p[i] = p[i-1]+dk-k*dl;
        q[i] = q[i-1]+m2*p[i];
        s[i] = s[i-1]+q[i]+m1*p[i];
    for i in range(length):
        s[i] = s[i]/(0.5*shape_time*(shape_time+1)*m1);
    res = fft.rfft(s)
    return res[:-1*shape_time];

In [3]:
def fft_trapezoid(rise_time, flat_top , tau):
    length = wavelen
    length = length+2*rise_time+flat_top;
    p = zeros(length)
    s = zeros(length)
    input2 = zeros(length)
    p[0] = s[0] = input2[0] = 0.;
    for i in range(1,length):
        input2[i] = s[i] = 0.;
    input2[1]=1.;
    tau = 1/(exp(1./tau)-1);
    for i in range(1,length):
        if i>=2*rise_time+flat_top:
            d = input2[i]-input2[i-rise_time]-input2[i-rise_time-flat_top]+input2[i-2*rise_time-flat_top]
        else:
            if i>=rise_time+flat_top:
                d = input2[i]-input2[i-rise_time]-input2[i-rise_time-flat_top]
            else:
                if i>=rise_time:
                    d = input2[i]-input2[i-rise_time]
                else:
                    d = input2[i];
        p[i] = p[i-1]+d;
        s[i] = s[i-1]+p[i]+tau*d;
    for i in range(length):
        s[i] = s[i]/(rise_time*tau);
    
    res = fft.rfft(s)